##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Migrar métricas y optimizadores

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/metrics_optimizers"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/metrics_optimizers.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/metrics_optimizers.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/migrate/metrics_optimizers.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

En TF1, `tf.metrics` es el namespace de la API para todas las funciones de métricas. Cada una de las métricas es una función que toma `label` y `prediction` como parámetros de entrada y devuelve como resultado el tensor de métricas correspondiente. En TF2, `tf.keras.metrics` contiene todas las funciones y objetos de métricas. El objeto `Metric` puede usarse con `tf.keras.Model` y `tf.keras.layers.layer` para calcular valores de métricas.

## Preparación

Empecemos con un par de imports TensorFlow necesarios,

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

y prepara algunos datos sencillos para la demostración:

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [0, 0, 1]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [0, 1, 1]

## TF1: tf.compat.v1.metrics con Estimator

En TF1, las métricas pueden añadirse a `EstimatorSpec` como `eval_metric_ops`, y la operación se genera mediante todas las funciones de métricas definidas en `tf.metrics`. Puedes seguir el ejemplo para ver cómo usar `tf.metrics.accuracy`.

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(2)(features)
  predictions = tf.math.argmax(input=logits, axis=1)
  loss = tf1.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  accuracy = tf1.metrics.accuracy(labels=labels, predictions=predictions)
  return tf1.estimator.EstimatorSpec(mode, 
                                     predictions=predictions,
                                     loss=loss, 
                                     train_op=train_op,
                                     eval_metric_ops={'accuracy': accuracy})

estimator = tf1.estimator.Estimator(model_fn=_model_fn)
estimator.train(_input_fn)

In [ ]:
estimator.evaluate(_eval_input_fn)

También se pueden añadir métricas al estimador directamente mediante `tf.estimator.add_metrics()`.

In [ ]:
def mean_squared_error(labels, predictions):
  labels = tf.cast(labels, predictions.dtype)
  return {"mean_squared_error": 
          tf1.metrics.mean_squared_error(labels=labels, predictions=predictions)}

estimator = tf1.estimator.add_metrics(estimator, mean_squared_error)
estimator.evaluate(_eval_input_fn)

## TF2: API de métricas Keras con tf.keras.Model

En TF2, `tf.keras.metrics` contiene todas las clases y funciones de métricas. Están diseñadas con un estilo OOP y se integran estrechamente con otras APIs `tf.keras`. Todas las métricas se pueden encontrar en el namespace `tf.keras.metrics`, y suele haber un mapeo directo entre `tf.compat.v1.metrics` con `tf.keras.metrics`.

En el siguiente ejemplo, las métricas se añaden en el método `model.compile()`. Los usuarios sólo tienen que crear la instancia de métricas, sin especificar la etiqueta ni el tensor de predicción. El modelo Keras dirigirá la salida del modelo y la etiqueta al objeto métrica.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

inputs = tf.keras.Input((2,))
logits = tf.keras.layers.Dense(2)(inputs)
predictions = tf.math.argmax(input=logits, axis=1)
model = tf.keras.models.Model(inputs, predictions)
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer, loss='mse', metrics=[tf.keras.metrics.Accuracy()])

In [ ]:
model.evaluate(eval_dataset, return_dict=True)

Con la ejecución eager activada, las instancias `tf.keras.metrics.Metric` pueden usarse directamente para evaluar datos numpy o tensores eager. Los objetos `tf.keras.metrics.Metric` son contenedores con estado. El valor de la métrica puede actualizarse mediante `metric.update_state(y_true, y_pred)`, y el resultado puede recuperarse mediante `metrics.result()`.


In [ ]:
accuracy = tf.keras.metrics.Accuracy()

accuracy.update_state(y_true=[0, 0, 1, 1], y_pred=[0, 0, 0, 1])
accuracy.result().numpy()


In [ ]:
accuracy.update_state(y_true=[0, 0, 1, 1], y_pred=[0, 0, 0, 0])
accuracy.update_state(y_true=[0, 0, 1, 1], y_pred=[1, 1, 0, 0])
accuracy.result().numpy()

Para obtener más información sobre `tf.keras.metrics.Metric`, consulta la documentación de la API en `tf.keras.metrics.Metric`, así como la [guía de migración](https://www.tensorflow.org/guide/effective_tf2#new-style_metrics_and_losses).

## Migrar optimizadores TF1.x a optimizadores Keras

Los optimizadores de `tf.compat.v1.train`, como el optimizador [Adam](https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/AdamOptimizer) y el [optimizador de descenso gradiente](https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/GradientDescentOptimizer), tienen equivalentes en `tf.keras.optimizers`.

La tabla siguiente resume cómo puedes convertir estos optimizadores heredados en sus equivalentes de Keras. Puedes reemplazar directamente la versión TF1.x por la versión TF2, a menos que se requieran pasos adicionales (como [actualizar la tasa de aprendizaje predeterminada](../../guide/effective_tf2.ipynb#optimizer_defaults)).

Ten en cuenta que convertir tus optimizadores [puede hacer incompatibles los puntos de verificación previos](./migrating_checkpoints.ipynb).

<table>
  <tr>
    <th>TF1.x</th>
    <th>TF2</th>
    <th>Pasos adicionales</th>
  </tr>
  <tr>
    <td>`tf.v1.train.GradientDescentOptimizer`</td>
    <td>`tf.keras.optimizers.SGD`</td>
    <td>Ninguno</td>
  </tr>
  <tr>
    <td>`tf.v1.train.MomentumOptimizer`</td>
    <td>`tf.keras.optimizers.SGD`</td>
    <td>Incluye el argumento "momentum"</td>
  </tr>
  <tr>
    <td>`tf.v1.train.AdamOptimizer`</td>
    <td>`tf.keras.optimizers.Adam`</td>
    <td>Cambia el nombre de los argumentos "beta1" y "beta2" a "beta_1" y "beta_2".</td>
  </tr>
  <tr>
    <td>`tf.v1.train.RMSPropOptimizer`</td>
    <td>`tf.keras.optimizers.RMSprop`</td>
    <td>Cambia el nombre del argumento "decay" por "rho".</td>
  </tr>
  <tr>
    <td>`tf.v1.train.AdadeltaOptimizer`</td>
    <td>`tf.keras.optimizers.Adadelta`</td>
    <td>Ninguno</td>
  </tr>
  <tr>
    <td>`tf.v1.train.AdagradOptimizer`</td>
    <td>`tf.keras.optimizers.Adagrad`</td>
    <td>Ninguno</td>
  </tr>
  <tr>
    <td>`tf.v1.train.FtrlOptimizer`</td>
    <td>`tf.keras.optimizers.Ftrl`</td>
    <td>Elimina los argumentos "accum_name" y "linear_name"</td>
  </tr>
  <tr>
    <td>`tf.contrib.AdamaxOptimizer`</td>
    <td>`tf.keras.optimizers.Adamax`</td>
    <td>Cambia el nombre de los argumentos "beta1" y "beta2" a "beta_1" y "beta_2".</td>
  </tr>
  <tr>
    <td>`tf.contrib.Nadam`</td>
    <td>`tf.keras.optimizers.Nadam`</td>
    <td>Cambia el nombre de los argumentos "beta1" y "beta2" a "beta_1" y "beta_2".</td>
  </tr>
</table>

Nota: En TF2, todos los épsilones (constantes numéricas de estabilidad) ahora están predeterminados a `1e-7` en lugar de `1e-8`. Esta diferencia es insignificante en la mayoría de los casos que se usan.